## Members
- Cao Thanh Khiết   19120544
- Ninh Duy Huy      19120533
- Lê Minh Hữu       19120525
- Nguyễn Tuấn Khoa  19120547
- Trần Tuấn Kiệt    19120557
# Task of each member
- Cao Thanh Khiết
    - Use SoundClound API to get data and write down them into csv file
- Ninh Duy Huy
    - Parse HTML to get data and write down them into csv file
- Lê Minh Hữu
    - Parse HTML to get data and write down them into csv file
- Nguyễn Tuấn Khoa
    - Parse HTML to get data and write down them into csv file
- Trần Tuấn Kiệt
    - Parse HTML to get data and write down them into csv file

In [1]:
import sys
sys.executable

'D:\\exec-in-cmd\\miniconda3\\envs\\min_ds-env\\python.exe'

In [2]:
from bs4 import BeautifulSoup
import time
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
from datetime import datetime, timedelta # Dùng để xử lý dữ liệu thời gian
import os
from selenium import webdriver
from concurrent import futures

In [3]:
def GetAllUrls(filename):
    urls = []
    with open(filename, 'r') as f:
        while f.tell() != os.fstat(f.fileno()).st_size:
            url = f.readline()
            if url != '':
                url = url.replace('\n','')
                urls.append(url)
    f.close()
    return urls

In [4]:
filename = 'url_entry/top50_url.txt'
urls = GetAllUrls(filename)

In [5]:
Users = []
Users.append([])
Users.append([])

def run_get_users(url):
    main_url = 'https://soundcloud.com'
    UsersLink = []
    UsersName = []
    
    chrome_options = webdriver.ChromeOptions()
    prefs = {
        "profile.managed_default_content_settings.images": 2
    }
    chrome_options.add_experimental_option("prefs", prefs)
    
    browser = webdriver.Chrome(executable_path="chromedriver.exe", options=chrome_options)
    browser.get(url)
    # To make it load more scroll to the bottom of the page (repeat if you want to)
    lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match = False
    while(match == False):
        lastCount = lenOfPage
        time.sleep(1)
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        if lastCount == lenOfPage:
                match = True

    html_source = browser.page_source
    browser.quit()
    soup = BeautifulSoup(html_source, 'html.parser')
    for id in soup.findAll('a', {'class' : 'trackItem__username sc-link-light sc-link-secondary'}):
        UsersLink.append(main_url + str(id.get('href')))
        UsersName.append(id.text)
    
    #đoạn này nối chuỗi mới vào Users[0], Users[1], Users[2]
    Users[0] += UsersLink 
    Users[1] += UsersName

with futures.ThreadPoolExecutor(max_workers=40) as executor:
    executor.map(run_get_users, urls)

In [6]:
# print(Users)
Users_new = []
for index, link in enumerate(Users[0]):
    Users_new.append([link, Users[1][index]])

In [7]:
TracksOfAllUsers = []
TracksOfAllUsers.append([]) #tên mỗi user
TracksOfAllUsers.append([]) #tên mỗi track của user
TracksOfAllUsers.append([]) #lấy thời gian post
TracksOfAllUsers.append([]) #lấy lượt nghe
TracksOfAllUsers.append([]) #lấy lượt thich

def run_get_tracks(user):   # user contains [link, user_name]
    UserName = []
    TrackName = []
    PostTime = []
    Plays = []
    Likes = []
    #----------------------------
    url = str(user[0]) + '/tracks'
    chrome_options = webdriver.ChromeOptions()
    prefs = {
        "profile.managed_default_content_settings.images": 2
    }
    chrome_options.add_experimental_option("prefs", prefs)
    browser = webdriver.Chrome(executable_path="chromedriver.exe", options=chrome_options)
    browser.get(url)
    # To make it load more scroll to the bottom of the page (repeat if you want to)        
    lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match = False
    while(match == False):
        lastCount = lenOfPage
        time.sleep(1)
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        if lastCount == lenOfPage:
            match = True
    html_source = browser.page_source
    browser.quit()
    
    soup = BeautifulSoup(html_source, 'html.parser')
    check = soup.findAll('div', {'class' : 'sound__content'})
    if check == []:
        UserName.append(user[1])
        TrackName.append('None')
        PostTime.append('None')
        Plays.append('None')
        Likes.append('None')
        
        TracksOfAllUsers[0] += UserName
        TracksOfAllUsers[1] += TrackName
        TracksOfAllUsers[2] += PostTime
        TracksOfAllUsers[3] += Plays
        TracksOfAllUsers[4] += Likes
    else:
        for id in check:
            UserName = []
            TrackName = []
            PostTime = []
            Plays = []
            Likes = []
    
            UserName.append(user[1])
        
            trackname = id.find('a', {'class' : 'sc-link-primary soundTitle__title sc-link-dark sc-text-h4'})
            Like = id.find('button', {'class' : 'sc-button-like sc-button-secondary sc-button sc-button-small sc-button-responsive'})
            info = id.findAll('span', {'class' : 'sc-visuallyhidden'})
        
            TrackName.append(trackname.text.replace('\n', ''))
            PostTime.append(info[0].text.replace('\n', '')) #lay thoi gian post
            
            if len(info) < 2:
                Plays.append('0')
                Likes.append('0')
            else:
                Plays.append(info[1].text.replace('\n', '')) #lay luot xem
                Likes.append(Like.text.replace('\n', ''))
                
            TracksOfAllUsers[0] += UserName
            TracksOfAllUsers[1] += TrackName
            TracksOfAllUsers[2] += PostTime
            TracksOfAllUsers[3] += Plays
            TracksOfAllUsers[4] += Likes

with futures.ThreadPoolExecutor(max_workers=20) as executor:
    executor.map(run_get_tracks, Users_new)

In [8]:
Users.append([]) #append follower column
Users.append([]) #append following column
Users.append([]) #append tracks column

def run_get_userinfor(user):
    url = user[0] #lay duong dan den user    
    chrome_options = webdriver.ChromeOptions()
    prefs = {
        "profile.managed_default_content_settings.images": 2
    }
    chrome_options.add_experimental_option("prefs", prefs)
    browser = webdriver.Chrome(executable_path="chromedriver.exe", options=chrome_options)
    browser.get(url)
    # xóa phần cuộn đến cuối trang
    html_source = browser.page_source
    browser.quit()

    soup = BeautifulSoup(html_source, 'html.parser')
    checks = soup.find_all('div', {'class' : 'infoStats__value sc-font-light'})
    Users[2].append(checks[0].text)
    Users[3].append(checks[1].text)
    Users[4].append(checks[2].text)
        
with futures.ThreadPoolExecutor(max_workers=20) as executor:
    executor.map(run_get_userinfor, Users_new)

In [ ]:
PlaylistOfAllUsers = []
PlaylistOfAllUsers.append([])
PlaylistOfAllUsers.append([])
PlaylistOfAllUsers.append([])

def run_get_playlists(user):
    UserNames = []
    PlaylistNames = []
    TrackNames = []
    
    url = str(user[0]) + '/sets' #lay duong dan den playlist
    chrome_options = webdriver.ChromeOptions()
    prefs = {
        "profile.managed_default_content_settings.images": 2
    }
    chrome_options.add_experimental_option("prefs", prefs)
    browser = webdriver.Chrome(executable_path="chromedriver.exe", options=chrome_options)
    browser.get(url)

    # To make it load more scroll to the bottom of the page (repeat if you want to)
    lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match = False
    while(match == False):
        lastCount = lenOfPage
        time.sleep(1)
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        if lastCount == lenOfPage:
            match = True    
    html_source = browser.page_source
    browser.quit()
    
    soup = BeautifulSoup(html_source, 'html.parser')
    check = soup.findAll('div', {'class' : 'sound__content'})
    if check == []:
        UserNames.append(user[1])
        PlaylistNames.append('None')
        TrackNames.append('None')
        
        PlaylistOfAllUsers[0] += UserNames
        PlaylistOfAllUsers[1] += PlaylistNames
        PlaylistOfAllUsers[2] += TrackNames
    else:        
        for id in check:
            Title = id.find('a', {'class' : 'sc-link-primary soundTitle__title sc-link-dark sc-text-h4'})
            Tracks = id.findAll('span', {'class' : 'compactTrackListItem__trackTitle sc-text-primary sc-text-h4'})
            for Track in Tracks:
                UserNames = []
                PlaylistNames = []
                TrackNames = []
                
                UserNames.append(user[1])
                PlaylistNames.append(Title.text.replace('\n', ''))
                TrackNames.append(Track.text.replace('\n', ''))
                
                PlaylistOfAllUsers[0] += UserNames
                PlaylistOfAllUsers[1] += PlaylistNames
                PlaylistOfAllUsers[2] += TrackNames

with futures.ThreadPoolExecutor(max_workers=20) as executor:
    executor.map(run_get_playlists, Users_new)

In [ ]:
def writeUsersToCsv(List5D, filename):
    df = pd.DataFrame({
        'Users Link': pd.Series(link for link in List5D[0]),
        'User Name': pd.Series(name for name in List5D[1]),
        'Followers': pd.Series(fle for fle in List5D[2]),
        'Following': pd.Series(fli for fli in List5D[3]),
        'Tracks': pd.Series(track for track in List5D[4]),
    })
    df.to_csv(filename, index = False)
    
def writeTracksToCsv(List5D, filename):
    df = pd.DataFrame({
        'User Name': pd.Series(user for user in List5D[0]),
        'Track Name': pd.Series(track for track in List5D[1]),
        'Post Time' : pd.Series(t for t in List5D[2]),
        'Plays' : pd.Series(p for p in List5D[3]),
        'Likes' : pd.Series(L for L in List5D[4]),
    })
    df.to_csv(filename, index = False)

def writePlaylistsToCsv(List3D, filename):
    df = pd.DataFrame({
        'User Name': pd.Series(user for user in List3D[0]),
        'Playlist Name': pd.Series(p for p in List3D[1]),
        'Track Name' : pd.Series(t for t in List3D[2]),
    })
    df.to_csv(filename, index = False)

In [ ]:
writeUsersToCsv(Users, 'Crawl_data/users.csv')
writeTracksToCsv(TracksOfAllUsers, './Crawl_data/tracks.csv')
writePlaylistsToCsv(PlaylistOfAllUsers, './Crawl_data/playlists.csv')